# 번역 함수

google translate api를 사용했으나 오역이 많아, open api로 프롬프트 생성하여 수행

In [1]:
import openai

In [3]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

In [4]:
def print_streaming_response(response):
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="")

In [5]:
messages = [{"role": "user", "content": "안녕하세요"}]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)


In [22]:
def request_chat_completion(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "당신은 한국어 식당 리뷰를 번역하는 영어번역가입니다."},
            {"role": "user", "content": prompt}
        ],
        stream = False
    )
    return response.choices[0].message.content

def generate_prompt(review):
    prompt = f"""
Translate the following Korean restaurant review into natural English:
{review}
""".strip()
    return prompt


# 리뷰를 번역하여 데이터프레임에 할당하고 저장하는 함수
def translate_review(file):
    file_path = f"./data/{file}.csv"
    
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding="utf-8")

    # "translated_review" 컬럼이 없다면 추가
    if "translated_review" not in df.columns:
        df["translated_review"] = ""

    # 각 리뷰에 대해 번역 수행
    for i in range(len(df)):
        review = df.loc[i, "review_text"]  # 리뷰 컬럼이 "review_text"라고 가정
        prompt = generate_prompt(review)  # 번역을 위한 프롬프트 생성
        translated_review = request_chat_completion(prompt)  # 번역 수행
        df.loc[i, "translated_review"] = translated_review  # 번역 결과 추가

    # 번역된 데이터프레임을 새로운 파일로 저장
    output_path = f"./data/translated_{file}.csv"
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"번역 완료: {file} -> {output_path}")
    return df

# 번역할 CSV 파일들의 리스트
file_names = ['신촌2호점',
              '우장산역점',
              '성신여대점',
              '교대역점',
              '관악난곡점',
              '목동남부시장점',
              '홍대입구역점',
              '구로디지털단지역점',
              '수유역점',
              '이수역점',
              '신풍점',
              '약수점',
              '대흥역점',
              '강서홈플러스점']

# 모든 파일에 대해 번역 수행
for file_name in file_names:
    translate_review(file_name)


번역 완료: 신촌2호점 -> ./data/translated_신촌2호점.csv
번역 완료: 우장산역점 -> ./data/translated_우장산역점.csv
번역 완료: 성신여대점 -> ./data/translated_성신여대점.csv
번역 완료: 교대역점 -> ./data/translated_교대역점.csv
번역 완료: 관악난곡점 -> ./data/translated_관악난곡점.csv
번역 완료: 목동남부시장점 -> ./data/translated_목동남부시장점.csv
번역 완료: 홍대입구역점 -> ./data/translated_홍대입구역점.csv
번역 완료: 구로디지털단지역점 -> ./data/translated_구로디지털단지역점.csv
번역 완료: 수유역점 -> ./data/translated_수유역점.csv
번역 완료: 이수역점 -> ./data/translated_이수역점.csv
번역 완료: 신풍점 -> ./data/translated_신풍점.csv
번역 완료: 약수점 -> ./data/translated_약수점.csv
번역 완료: 대흥역점 -> ./data/translated_대흥역점.csv
번역 완료: 강서홈플러스점 -> ./data/translated_강서홈플러스점.csv
